####  NOTES
>1. Verified that the model class and original model definition via repeated functio calls have extremely close validation scores **(0.2998 for normal and 0.3042 class for class)** ie, they are probably same.
>2. Added better code cleaning functionalities and mis-spelling corrections.


####  TODO
>1. There is text augmenter library https://github.com/makcedward/nlpaug/blob/master/example/textual_augmenter.ipynb

# Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
! cp -r drive/My\ Drive/ml_projects/kaggle/google_quest/google-quest-challenge .

In [0]:
# !ls  drive/My\ Drive/ml_projects/ml_datasets/nlp_pretrained/gensim-embeddings-dataset

In [0]:
! cp drive/My\ Drive/ml_projects/ml_datasets/nlp_pretrained/gensim-embeddings-dataset/glove.twitter.27B.200d.gensim .
! cp drive/My\ Drive/ml_projects/ml_datasets/nlp_pretrained/gensim-embeddings-dataset/glove.twitter.27B.200d.gensim.vectors.npy .

In [1]:
! pip install iterative-stratification

# Start

In [0]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.run_this_cell {display: block !important;} </style"))

In [2]:
%tensorflow_version 2.x 

TensorFlow 2.x selected.


In [0]:
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, concatenate, LSTM, Bidirectional, Embedding, add, Dense, Masking
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback

from tensorflow.keras import models, layers, optimizers


from datetime import datetime
from typing import List, Dict, Tuple, Any

from collections import Counter
from tqdm import trange, tqdm, tnrange, tqdm_notebook
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

In [0]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit, MultilabelStratifiedKFold

In [0]:
np.random.seed(42)
tf.random.set_seed(42)

In [5]:
tf.__version__

'2.1.0'

In [6]:
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

In [0]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

### Utility functions

In [0]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '\xa0', '\t',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '\u3000', '\u202f',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '«',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

In [0]:
def clean_text(x):

    x = str(x)
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^`{|}~' + '“”’':
        x = x.replace(punct, f' {punct}')
   
    for punct in '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—_':
        x = x.replace(punct, f' {punct}')
        
    for punct in puncts:
        x = x.replace(punct, f' {punct}')
 
    return x

In [0]:
def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

In [0]:
mispell_dict = {"usepackage" : "use package",
                'instrumentsettingsid':'instrumental settings id',
                'RippleShaderProgram' : 'ripple shader program',
                'ShaderProgramConstants':'shader program constants',
                'storedElements':'stored elements',
                'stackSize' : 'stack size',
                '_':' ',
                "aren't" : "are not",
                "can't" : "cannot",
                "couldn't" : "could not",
                "couldnt" : "could not",
                "didn't" : "did not",
                "doesn't" : "does not",
                "doesnt" : "does not",
                "don't" : "do not",
                "hadn't" : "had not",
                "hasn't" : "has not",
                "haven't" : "have not",
                "havent" : "have not",
                "he'd" : "he would",
                "he'll" : "he will",
                "he's" : "he is",
                "i'd" : "I would",
                "i'd" : "I had",
                "i'll" : "I will",
                "i'm" : "I am",
                "isn't" : "is not",
                "it's" : "it is",
                "it'll":"it will",
                "i've" : "I have",
                "let's" : "let us",
                "mightn't" : "might not",
                "mustn't" : "must not",
                "shan't" : "shall not",
                "she'd" : "she would",
                "she'll" : "she will",
                "she's" : "she is",
                "shouldn't" : "should not",
                "shouldnt" : "should not",
                "that's" : "that is",
                "thats" : "that is",
                "there's" : "there is",
                "theres" : "there is",
                "they'd" : "they would",
                "they'll" : "they will",
                "they're" : "they are",
                "theyre":  "they are",
                "they've" : "they have",
                "we'd" : "we would",
                "we're" : "we are",
                "weren't" : "were not",
                "we've" : "we have",
                "what'll" : "what will",
                "what're" : "what are",
                "what's" : "what is",
                "what've" : "what have",
                "where's" : "where is",
                "who'd" : "who would",
                "who'll" : "who will",
                "who're" : "who are",
                "who's" : "who is",
                "who've" : "who have",
                "won't" : "will not",
                "wouldn't" : "would not",
                "you'd" : "you would",
                "you'll" : "you will",
                "you're" : "you are",
                "you've" : "you have",
                "'re": " are",
                "wasn't": "was not",
                "we'll":" will",
                "didn't": "did not",
                "tryin'":"trying"}

In [0]:
mispellings, mispellings_re = _get_mispell(mispell_dict)

In [0]:
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)

### Loading Data

In [0]:
# train_loc = '/kaggle/input/google-quest-challenge/train.csv'
# test_loc= '/kaggle/input/google-quest-challenge/test.csv'

# train_loc = '../google-quest-challenge/train.csv'
# test_loc= '../google-quest-challenge/test.csv'

train_loc = 'google-quest-challenge/train.csv'
test_loc= 'google-quest-challenge/test.csv'

train_df = pd.read_csv(train_loc)
test_df = pd.read_csv(test_loc)


### Preprocessing

In [0]:
train_df["question_body"] = train_df["question_body"].apply(clean_text)
train_df["question_body"] = train_df["question_body"].apply(replace_typical_misspell)

train_df["question_title"] = train_df["question_title"].apply(clean_text)
train_df["question_title"] = train_df["question_title"].apply(replace_typical_misspell)

train_df["category"] = train_df["category"].apply(clean_text)
train_df["category"] = train_df["category"].apply(replace_typical_misspell)

train_df["answer"] = train_df["answer"].apply(clean_text)
train_df["answer"] = train_df["answer"].apply(replace_typical_misspell)


test_df["question_body"] = test_df["question_body"].apply(clean_text)
test_df["question_body"] = test_df["question_body"].apply(replace_typical_misspell)

test_df["question_title"] = test_df["question_title"].apply(clean_text)
test_df["question_title"] = test_df["question_title"].apply(replace_typical_misspell)

test_df["category"] = test_df["category"].apply(clean_text)
test_df["category"] = test_df["category"].apply(replace_typical_misspell)

test_df["answer"] = test_df["answer"].apply(clean_text)
test_df["answer"] = test_df["answer"].apply(replace_typical_misspell)

In [0]:
question_body = train_df['question_body']
answer = train_df['answer']
question_title = train_df["question_title"]
category = train_df["category"]

question_body_test = test_df['question_body']
answer_test = test_df['answer']
question_title_test = test_df["question_title"]
category_test = test_df["category"]

In [0]:
target = train_df[train_df.columns[-30:]]

In [0]:
target_cols = train_df.columns[-30:]

### Creating Tokenziers

In [0]:
all_text = pd.concat([
                    train_df['question_body'],
                     train_df['answer'],
                     test_df['question_body'],
                     test_df['answer'],
                     train_df["question_title"],
                     train_df["category"],
                     test_df["question_title"],
                     test_df["category"]
                    ])

In [20]:
len(train_df), len(test_df), 6079*4 + 476 * 4, len(all_text)

(6079, 476, 26220, 26220)

In [0]:
tokenizer = Tokenizer(num_words=1000000, lower=False,filters='')

tokenizer.fit_on_texts(all_text)

In [22]:
len(tokenizer.word_counts), tokenizer.word_index['the'], tokenizer.word_index['ABC']

(100498, 4, 8783)

In [0]:
question_body = tokenizer.texts_to_sequences(question_body)
answer = tokenizer.texts_to_sequences(answer)
question_title = tokenizer.texts_to_sequences(question_title)
category = tokenizer.texts_to_sequences(category)

question_body_test = tokenizer.texts_to_sequences(question_body_test)
answer_test = tokenizer.texts_to_sequences(answer_test)
question_title_test = tokenizer.texts_to_sequences(question_title_test)
category_test = tokenizer.texts_to_sequences(category_test)

In [24]:
len(question_body), len(question_body[0])

(6079, 174)

In [0]:
vocab_size = len(tokenizer.word_index) + 1
maxlen = 245

### Padding

In [0]:
question_body = pad_sequences(question_body, padding='post', maxlen=maxlen)
answer = pad_sequences(answer, padding='post', maxlen=maxlen)
question_title = pad_sequences(question_title, padding='post', maxlen=maxlen)
category = pad_sequences(category, padding='post', maxlen=maxlen)


question_body_test = pad_sequences(question_body_test, padding='post', maxlen=maxlen)
answer_test = pad_sequences(answer_test, padding='post', maxlen=maxlen)
question_title_test = pad_sequences(question_title_test, padding='post', maxlen=maxlen)
category_test = pad_sequences(category_test, padding='post', maxlen=maxlen)

### Build embedding matrix for the full dataset

Gensim word vectors can be found here https://www.kaggle.com/iezepov/gensim-embeddings-dataset

All UNK tokens will be 0s. This is different from the NLP project, where the UNK tokens are the average of the matrix.

In [0]:
emb_size = 200

def build_matrix(word_index, path):
    embedding_index = KeyedVectors.load(path, mmap='r')
    embedding_matrix = np.zeros((len(word_index) + 1, emb_size))
    for word, i in word_index.items():
        for candidate in [word, word.lower()]:
            if candidate in embedding_index:
                embedding_matrix[i] = embedding_index[candidate]
                break
    return embedding_matrix

In [0]:
# EMBEDDING_FILES = ['/Users/emiljoswin/study/ml-projects/NLP/pre-trained-weights/glove/glove.840B.300d.txt']

# glove_file = datapath(EMBEDDING_FILES[0])

# tmp_file = get_tmpfile("test_word2vec.txt")

# _ = glove2word2vec(glove_file, "test_word2vec.txt")

# model = KeyedVectors.load_word2vec_format(tmp_file)


In [28]:
# EMBEDDING_FILES = ['/Users/emiljoswin/study/playground/jupyter_notebooks/NLP/crawl-300d-2M.gensim', 
#                   '/Users/emiljoswin/study/playground/jupyter_notebooks/NLP/glove.840B.300d.gensim']

# EMBEDDING_FILES = ['/kaggle/input/gensim-word-embeddings/glove.twitter.27B.200d.gensim']

EMBEDDING_FILES = ['glove.twitter.27B.200d.gensim']


# EMBEDDING_FILES = ['../../NLP/pre-trained-weights/gensim-embeddings-dataset/glove.twitter.27B.200d.gensim']
# EMBEDDING_FILES = ['/Users/emiljoswin/study/ml-projects/NLP/pre-trained-weights/gensim-embeddings-dataset/glove.840B.300d.gensim']


embedding_index = KeyedVectors.load(EMBEDDING_FILES[0], mmap='r')
# embedding_index = KeyedVectors.load(EMBEDDING_FILES[1], mmap='r')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [29]:
m1 = build_matrix(tokenizer.word_index, EMBEDDING_FILES[0])
# m2 = build_matrix(tokenizer.word_index, EMBEDDING_FILES[1])
# m1.shape, m2.shape, 
m1.shape

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


(100499, 200)

In [30]:
# embedding_matrix = np.concatenate([m1, m2], axis=-1)
# embedding_matrix.shape

embedding_matrix = m1
embedding_matrix.shape

(100499, 200)

# Train Test Split

In [31]:
X = concatenate([question_body, question_title, category, answer], 1)
print(X.shape)
Y = target.to_numpy()
print(Y.shape)

test_original = concatenate([question_body_test, question_title_test, category_test, answer_test], 1)
print(test_original.shape)

(6079, 980)
(6079, 30)
(476, 980)


In [32]:
total = 6079
val_size = 700

p = np.random.permutation(list(range(total)))

val_idx = p[:val_size]
train_idx = p[val_size:]

train_x = tf.gather(X, train_idx)
train_y = tf.gather(Y, train_idx)

val_x = tf.gather(X, val_idx)
val_y = tf.gather(Y, val_idx)

print(train_x.shape, train_y.shape, val_x.shape, val_y.shape)


(5379, 980) (5379, 30) (700, 980) (700, 30)


In [33]:
print(val_idx)

[ 288 2811 4785 4418 2758 5392 4497 4225 3837 3285 3834 5151 6077 2574
 2270 2820 1345  653 2673  855 4191 4250 2951 3494   80 4967 4104  400
 1761 3482  718 5350 5584 2032 3443 3038 4773 1127 5157 4700 5721 2053
 1358  319  209 5731 1155  238 1614 4185 2174 3422 5404 2659 4883 2865
 1487 4951 1543 4887 4596 4871 3300 2395 4812  113 1486 4993 3791 2036
 3520 3832 2821 3857 2809 5411 5365 5158 5409 2976  975 3603 4333  945
 1716 6035 4114  432  520 5487 2099 1575 2251 1298 1413 2166 1060  483
  293 5491 2285 3064  442 4904 3729 4448 5918 5597 2170  670 3724 1444
 3546 5028  395 1685 4262 5309 3806 2078 1106 2601 3307  281 5471  572
 2165 4463 5980 2076 4532 5210 2812 5454 2204 2539 4390 4243 4394 3697
 3170 1872 5698 1343  360 4576 2714 2766 5375 3316   95 4275 4663 4401
  514 2264 1221 3394 3461 3111 1363  335  611 5178  986 4220 5542 5593
 3485 3351 4828 4896  423 2410 3741 4359 4382 1812 5133 2469   86 3110
 5783 5797 5059 4314 1617 5223 2253  321 1905 1253 2496 2411 5569 3921
 2180 

In [0]:
val_x = tf.convert_to_tensor(val_x)
val_y = tf.convert_to_tensor(val_y)

train_x = tf.convert_to_tensor(train_x)
train_y = tf.convert_to_tensor(train_y)

In [35]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape

(TensorShape([5379, 980]),
 TensorShape([5379, 30]),
 TensorShape([700, 980]),
 TensorShape([700, 30]))

# Evaluate

In [0]:
from scipy.stats import spearmanr
import math

def calculate_mean_spearman_co(a, b):
    """
        Calculates spearmans correlation coefficient 
        https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient
        
        In the scipy implementation if you pass x and y and if one of them is composed of all same values,
        then it returns nan. I am changing those nans to zeros
    """
    # print(a.shape, b.shape, len(a.shape))
    # print(a, b)
    if len(a.shape) == 1:
        return 0

    if a.shape[0] is None and a.shape[1] is None:
         return 0
         
    ans = [spearmanr(a[:, i], b[:, i])[0] for i in range(a.shape[1])]

    ans = [0. if math.isnan(elem) else elem for elem in ans]
    return np.mean(ans), ans

# Extras

In [0]:
# LSTM_UNITS = 64
# DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
# BATCH_SIZE = 128
# EPOCHS = 10

In [0]:
# words = Input(shape=(None,))

In [0]:
# x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
# x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(x)
# x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(x)

# hidden = concatenate([
#         GlobalMaxPooling1D()(x),
#         GlobalAveragePooling1D()(x),
#     ])

# hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
# hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])

In [0]:
# result = Dense(30, activation='sigmoid')(hidden)

In [0]:
# model2 = Model(inputs=words, outputs=[result])
# model2.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=['mae'])

In [0]:
# model2.summary()

In [0]:
# model2.fit(
#             train_x, train_y,
#             batch_size=256,
#             epochs=20,
#             verbose=1,
#         )

In [0]:
# val_predictions = model2.predict(val_x)
# m, _ = calculate_mean_spearman_co(val_predictions, val_y)
# print("val_spearman", m)

# Model as a class

In [0]:
LSTM_UNITS = 64
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
BSZ = 32
EPOCHS = 10
OUTPUT_CLASSES = 30

In [0]:
class MyModel(models.Model):
    def __init__(self, embedding_matrix):
        super(MyModel, self).__init__()
        self.mask = Masking(mask_value=0)
        self.emb = Embedding(*embedding_matrix.shape, weights=[embedding_matrix])
        self.b1 = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))
        self.b2 = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))
#         self.attn_weight = tf.Variable(tf.random.normal((LSTM_UNITS*2, 1)))
        self.g1 = GlobalMaxPooling1D()
        self.g2 = GlobalAveragePooling1D()
        
        self.d1 = Dense(DENSE_HIDDEN_UNITS, activation='relu')
        self.d2 = Dense(DENSE_HIDDEN_UNITS, activation='relu')
        self.linear = Dense(OUTPUT_CLASSES, activation='sigmoid')
        
    def attn(self, inputs):
        """
            inputs: [bsz x 980 x LSTM_UNITS*2]
            output: [bsz x LSTM_UNITS]
        """
        
        # (1 x 128) (bsz x seq x 128 )T  => (2 x seq)
        check = tf.einsum('ij,kjm->kim',
                          tf.transpose(self.attn_weight), 
                          tf.transpose(inputs, [0, 2, 1])) # (bsz x 1 x 980) - a score for each time step
        
        check = tf.squeeze(check) # (bsz x 980)
        check = tf.nn.softmax(check, axis=1) # attention weight (bsz x 980)
        
        #(bsz x seq x 128)T (bsz x seq) => (bsz x 128)
        check = tf.einsum('ijk,ki->ij', 
                          tf.transpose(inputs, [0, 2, 1]), 
                          tf.transpose(check) ) # (bsz x LSTM_UNITS) - best of each 980 time-steps
        
        return check

        
        
    def call(self, inputs):
#         msk = inputs != 0
        x = self.mask(inputs)
        x = self.emb(x)
        x = self.b1(x)
        x = self.b2(x)
#         x1 = self.attn(x) # (bsz x LSTM_UNITS*2)
#         x2 = self.attn(x)
        x = concatenate([self.g1(x), self.g2(x)])
        
        x = add([x, self.d1(x)])
        x = add([x, self.d2(x)])
        x = self.linear(x)
        return x

In [0]:
# model3 = MyModel(embedding_matrix)

In [0]:
# %%time 
# out = model3(train_x[0:2, :]) # Masking with Masking Class

# """
# CPU times: user 2min 33s, sys: 3.18 s, total: 2min 36s
# Wall time: 39.9 s
# """

In [0]:
# %%time 
# out = model3(train_x[0:2, :]) #Non masking
# """
# CPU times: user 2min 32s, sys: 2.68 s, total: 2min 35s
# Wall time: 38.9 s
# """

In [0]:
# %%time 
# out = model3(train_x[0:2, :]) #Masking normally by passing explicit mask values to LSTM layers
# """
# CPU times: user 3min 19s, sys: 3.48 s, total: 3min 22s
# Wall time: 53.9 s

# """

In [0]:
# out.shape

In [0]:
# model3.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=['mae'])

In [0]:
class SpearmanRhoCallback(Callback):
    def __init__(self, training_data, validation_data, patience, model_name):
        print("inside Init")
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]
        
        self.patience = patience
        self.value = -1
        self.bad_epochs = 0
        self.model_name = model_name

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return
    
    def on_epoch_end(self, epoch, logs={}):
        y_pred_val = self.model.predict(self.x_val)
#         rho_val = np.mean([spearmanr(self.y_val[:, ind], y_pred_val[:, ind] + np.random.normal(0, 1e-7, y_pred_val.shape[0])).correlation for ind in range(y_pred_val.shape[1])])
        rho_val = np.mean([ spearmanr(self.y_val[:, ind], y_pred_val[:, ind]).correlation for ind in range(y_pred_val.shape[1]) ])
        print("\n rho_val", rho_val)
        if rho_val >= self.value:
            self.value = rho_val
            self.bad_epochs = 0
            print('save best weights')
            self.model.save_weights(self.model_name)
        else:
            self.bad_epochs += 1
            print('bad epochs = ',self.bad_epochs)
        if self.bad_epochs >= self.patience:
            print("Epoch %05d: early stopping Threshold -- load best weights" % epoch)
            try:
                self.model.load_weights(self.model_name)
            except:
                print('could not load a model')
            self.model.stop_training = True
            
#             
        print('\rval_spearman-rho: %s' % (str(round(rho_val, 4))), end=100*' '+'\n')
        return rho_val

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

##### MISC code

In [0]:
# train_x.shape, val_x.shape

In [0]:
# NUM_FOLDS = 5
# kf = MultilabelStratifiedKFold(n_splits = NUM_FOLDS)

In [0]:
# Y.shape

In [0]:
# X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
# y = np.array([[0,0, 1], [0,0, 1], [0,1, 0], [0,1, 3], [1,1, 0], [1,1, 3], [1,0, 3], [1,0, 3]])

# mskf = MultilabelStratifiedKFold(n_splits=2)

# for train_index, test_index in mskf.split(X, y):
#    print("TRAIN:", train_index, "TEST:", test_index)
#    X_train, X_test = X[train_index], X[test_index]
#    y_train, y_test = y[train_index], y[test_index]

In [0]:
# NUM_FOLDS = 5
# kf = MultilabelStratifiedKFold(n_splits = NUM_FOLDS)

# for i, (train_idx, val_idx) in enumerate(kf.split(X, Y)):
#     print(len(train_idx), len(val_idx), train_idx[:5], val_idx[:5])

In [0]:
def create_model():
    model = MyModel(embedding_matrix)
    model.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=['mae'])
    return model

### CV

In [59]:
NUM_FOLDS = 3
kf = MultilabelStratifiedKFold(n_splits=NUM_FOLDS)
all_spearmans = []

def do_cv():
    for i, (train_idx, val_idx) in enumerate(kf.split(X, Y)):
        # print(len(train_idx), len(val_idx), train_idx[:5], val_idx[:5])
        print(f'Fold: {i+1}')
        model = create_model()
        train_x = tf.gather(X, train_idx)
        train_y = tf.gather(Y, train_idx)

        val_x = tf.gather(X, val_idx)
        val_y = tf.gather(Y, val_idx)

        # print( train_x.shape,  train_y.shape, val_x.shape, val_y.shape) 
        model.fit(
            x=train_x,
            y=train_y,
            batch_size=BSZ,
            epochs=1,
            verbose=1,
            validation_data=(val_x, val_y),
            callbacks=[SpearmanRhoCallback(training_data=(train_x, train_y), validation_data=(val_x, val_y),
                                            patience=15, model_name=f'best_model_batch_1.h5')]
        )

        pred_val = model.predict(val_x)
        spearmans = [ spearmanr(val_y[:, ind], pred_val[:, ind]).correlation for ind in range(pred_val.shape[1]) ]
        all_spearmans.append(spearmans)


do_cv()

Fold: 1
inside Init
Train on 4053 samples, validate on 2026 samples
4032/4053 [============================>.] - ETA: 0s - loss: 0.0584 - mae: 0.1692	 rho_val 0.23771471054174845

save best weights

val_spearman-rho: 0.2377                                                                                                    
4053/4053 [==============================] - 42s 10ms/sample - loss: 0.0584 - mae: 0.1691 - val_loss: 0.0537 - val_mae: 0.1562
Fold: 2
inside Init
Train on 4051 samples, validate on 2028 samples
4032/4051 [============================>.] - ETA: 0s - loss: 0.0585 - mae: 0.1689	 rho_val 0.23619459767448872

save best weights

val_spearman-rho: 0.2362                                                                                                    
4051/4051 [==============================] - 42s 10ms/sample - loss: 0.0584 - mae: 0.1688 - val_loss: 0.0528 - val_mae: 0.1594
Fold: 3
inside Init
Train on 4054 samples, validate on 2025 samples
4032/4054 [===================

#### Results to compare

In [60]:
m = np.mean(all_spearmans, axis=1)
print(f'Spearmans over folds: {m}')
std = np.std(m)
m = np.mean(m)
print(f'Overal Spearmans: {m} +/ {std}')

Spearmans over folds: [0.23771471 0.2361946  0.24517304]
Overal Spearmans: 0.2396941150338822 +/ 0.003923571865333851


In [61]:
m = np.mean(all_spearmans, axis=0)
print(f'Val score for each Labels')
print(f'`````````````````````````')
for i, t in enumerate(target_cols):
    print(f'{m[i]}  : {t} ')

Val score for each Labels
`````````````````````````
0.2549821114388325  : question_asker_intent_understanding 
0.379585228916246  : question_body_critical 
0.3182242617309056  : question_conversational 
0.19851628194693202  : question_expect_short_answer 
0.1306886057633775  : question_fact_seeking 
0.30197352716717746  : question_has_commonly_accepted_answer 
0.290024284462556  : question_interestingness_others 
0.41444200215857685  : question_interestingness_self 
0.2890225803891104  : question_multi_intent 
0.012557416492219843  : question_not_really_a_question 
0.21374984738715141  : question_opinion_seeking 
0.24161426329566202  : question_type_choice 
0.18333031674464842  : question_type_compare 
0.10015711014976263  : question_type_consequence 
0.25892575543800006  : question_type_definition 
0.22842744970598008  : question_type_entity 
0.6579571885157994  : question_type_instructions 
0.21829199879756941  : question_type_procedure 
0.38799984963538886  : question_type_reason_ex

### Final Model training

In [62]:
# After computing the total epoch number
model = create_model()
model.fit(
            X, tf.convert_to_tensor(Y),
            batch_size=BSZ,
            epochs=1,
            verbose=1,
        )

Train on 6079 samples
6079/6079 [==============================] - 46s 8ms/sample - loss: 0.0569 - mae: 0.1653


In [0]:
# val_predictions = model3.predict(val_x)
# m, _ = calculate_mean_spearman_co(val_predictions, val_y)
# print("val_spearman", m) #0.3237 => gave 0.289

# Saving the predictions

In [63]:
predictions = model.predict(test_original)
print(predictions.shape)

(476, 30)


In [0]:
# loc = '/kaggle/input/google-quest-challenge/sample_submission.csv'
loc = 'google-quest-challenge/sample_submission.csv'
sub = pd.read_csv(loc)

for col_index, col in enumerate(target_cols):
    sub[col] = predictions[:, col_index]

In [0]:
sub.to_csv('submission.csv', index = False)

# Overal Spearmans: 0.2396941150338822 +/ 0.003923571865333851